In [6]:
from oltk.dataset import VaultDataset
from oltk.features import extract_header_props
from oltk.utils.re_constants import JINJA_EXPR, TEMPLATER_EXPR
import re
import polars as pl

In [7]:
vault = VaultDataset("/root/igkh/codes/ObsidianLTK/examples/data/DataScienceNotes")

In [8]:
vault.construct()

In [15]:
samples = vault.get_v()

In [16]:
def tidy_up_template(content: str) -> str:
    content = re.sub(JINJA_EXPR, "__TEMPLATE_IMPUTATION__", content)
    content = re.sub(TEMPLATER_EXPR, "__TEMPLATE_IMPUTATION__", content)
    return content

In [17]:
samples = samples.with_columns(
    pl.col("text")
    .map_elements(
        tidy_up_template,
        return_dtype=str,
    )
    .alias("text_without_templates")
).drop(pl.col("text"))

In [ ]:
def extract_tags_and_date(x):
    props = extract_header_props(x)
    return {"tags": props.get("tags", []), "date": props.get("date", None)}


In [ ]:
samples = samples.with_columns(
    pl.col("text_without_templates")
    .map_elements(
        lambda x: extract_tags_and_date(x),
        # return_dtype=pl.List(str),
    )
    .alias("stat")
)

SchemaError: unexpected value while building Series of type List(String)

In [13]:
samples

id,title,text_without_templates,tags
u32,str,str,list[str]
0,"""Разные виды терминалов при под…","""--- tags: - DevOPS date: 202…","[""DevOPS""]"
1,"""Notes NER pipeline""","""--- tags: - NLP - obsidian…","[""NLP"", ""obsidian_meta""]"
2,"""Задача NNS - Nearest Neighbour…","""--- tags: - MathModeling -…","[""MathModeling"", ""Search""]"
3,"""Биномиальное Распределение""","""--- tags: - STATS layer: bro…","[""STATS""]"
4,"""RL парадигма""","""--- tags: - RL layer: bronze…","[""RL""]"
…,…,…,…
81,"""Иерархические обходы""","""--- tags: - WISDOM - Graph…","[""WISDOM"", ""Graphs"", ""GraphML""]"
82,"""Struc2Vec""","""--- tags: - Graphs - Graph…","[""Graphs"", ""GraphML""]"
83,"""Гипотеза структурной эквивален…","""--- tags: - Graphs - Graph…","[""Graphs"", ""GraphML""]"
